<a href="https://colab.research.google.com/github/Pmilivojevic/PyTorch/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.nn.modules.dropout import Dropout
from torch.nn.modules import activation

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
VGG16 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

In [ ]:
class VGG(nn.Module):
  def __init__(self, in_channels=3, num_classes=1000):
    super(VGG, self).__init__()
    self.in_channels = in_channels
    self.num_classes = num_classes

    self.conv_layers = self.create_conv_layers(VGG16)
    self.fcs = nn.Sequential(
        nn.Linear(in_features=512*7*7, out_features=4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fcs(x)

    return x

  def create_conv_layers(self, arhitecture):
    layers = []
    in_channels = self.in_channels

    for x in arhitecture:
      if type(x) == int:
        out_channels = x

        layers += [nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3,3),
            stride=(1,1),
            padding=(1,1),
          ),
            nn.BatchNorm2d(x),
            nn.ReLU()
        ]
        in_channels = x
      elif x == 'M':
        layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
    return nn.Sequential(*layers)

In [ ]:
model = VGG()
x = torch.randn(1, 3, 224, 224)
print(model(x).shape)

torch.Size([1, 1000])
